<a href="https://colab.research.google.com/github/konkuk-gaegul/MLP_Classification_AI/blob/main/MFCC_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import librosa
import copy, os, shutil
import matplotlib as plt

import matplotlib
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
# 폰트 설정을 위한 라이브러리 임포트
from matplotlib import font_manager, rcParams

# 한글 표현이 가능한 폰트를 설치
!apt-get install fonts-nanum*
font_manager.findSystemFonts(fontext='ttf')
font_manager.FontProperties( fname='/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf').get_name()

# 폰트를 변경
rcParams['font.family'] = 'NanumGothicCoding'

# - 부호도 깨져서 나오기 때문에, 같이 설정
rcParams['axes.unicode_minus'] = False

# 설정을 변경했으면, rebuild를 통해서 변경된 설정을 적용
font_manager._rebuild()

In [ ]:
# train data upload
train = pd.read_csv('mfcc_data_folder/train_mfcc_data_32.csv')
train.head(2)
test = pd.read_csv('mfcc_data_folder/test_mfcc_data_32.csv')
test.head(2)

,id,age,gender,respiratory_condition,fever_or_muscle_pain,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,3806,48,female,1,0,-619.9879,9.328649,0.821094,2.373343,-0.171163,...,-0.313271,-0.053711,-0.029855,0.043562,-0.405293,-0.246975,-0.110937,0.019196,-0.155829,-0.130705
1,3807,24,female,0,0,-493.6057,8.492651,-2.758262,-2.801817,-2.770231,...,-0.677968,-0.643835,-0.822677,-0.113844,-1.210391,0.273594,-0.685921,0.440101,-0.960277,0.371881


In [ ]:
# gender : OHE 처리
def onehot_encoding(ohe, x):
    # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
    encoded = ohe.transform(x['gender'].values.reshape(-1,1))
    encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
    x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
    x.drop( columns = 'id', inplace = True)
    return x

In [ ]:
# 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
ohe = OneHotEncoder(sparse=False)
ohe.fit(train['gender'].values.reshape(-1,1))

train = onehot_encoding(ohe, train)
test = onehot_encoding(ohe, test)

In [ ]:
# train_test_split
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train.drop(columns = 'covid19') , train.covid19 , test_size=0.3)

# print('학습시킬 train 셋 : ', train_X.shape, train_y.shape)
# print('검증할 val 셋 : ', test_X.shape, test_y.shape)

학습시킬 train 셋 :  (2663, 38) (2663,)
검증할 val 셋 :  (1142, 38) (1142,)
